# Using LangChain To Create ChatBot Via Konko

## Install necessary Python packages

In [1]:
# !pip install langchain
# !pip install konko

## Import Libraries

In [2]:
import konko
import os
from langchain.llms import Konko
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import initialize_agent

##  Set Environment Variables

Next, you need to set your environment variables for KONKO_URL and KONKO_TOKEN. Replace {your_token_here} with your actual token.

In [3]:
os.environ['KONKO_URL'] = "https://api.konko.ai/v0.3.0/"
os.environ['KONKO_TOKEN'] = "kYSoL8IyM28fpztmQYxib7hgkbjTdDGB2Icq4nM8"

## Set Up Your Language Model

Next, configure your language model. In this context, we'll be utilizing Konko as a provider within the langchain library, specifically using the 'llama-2-13b-chat' model.

In [4]:
llm = Konko(model_id='meta-llama--Llama-2-13b-chat-hf')

## Initializing an Conversational Agent


We first need to initialize the agent. Conversational agents require several things such as conversational memory, access to tools, and an llm (which we have already initialized).

In [5]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)
tools = load_tools(["llm-math"], llm=llm)

##  Define the OutputParser Class

Define the OutputParser class to handle the parsing of the model's output

In [6]:
class OutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> AgentAction | AgentFinish:
            return AgentFinish({"output": text}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"

# initialize output parser for agent
parser = OutputParser()

## Initialize the Agent

Next, initialize the agent with the defined parser, memory buffer, tools, and the Konko model

In [7]:
# initialize agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    agent_kwargs={"output_parser": parser}
)

# Set Up the Conversation Flow

Konko manages special instruction tokens, simplifying the creation of LLM-compatible prompts and streamlining your chatbot's interactions, allowing you to focus on other important aspects of your chatbot design.

In [8]:
human_msg = "{input}"

agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

## Now we can begin asking questions...

In [9]:
output = agent("Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool.")



> Entering new AgentExecutor chain...
 Hello! I'd be happy to help you with the crop tool in our photo editing app! 📸 Can you tell me a bit more about what you're trying to achieve with the crop tool? Are you looking to remove unwanted edges or objects from your photo, or do you want to adjust the composition of the image in some way? The more information you can provide, the better I'll be able to assist you. 😊

> Finished chain.


In [10]:
agent("I don't know how to adjust the size and position of the crop ")



> Entering new AgentExecutor chain...
 AI:  Sure thing! To adjust the size and position of the crop, you can use the handles that appear when you select an area to crop. The handles are the small squares that appear at the corners of the selection, and they allow you to adjust the size and position of the crop.

To adjust the size of the crop, simply click and drag one of the handles inward or outward to make the selection larger or smaller. To adjust the position of the crop, click and drag one of the handles to move the selection to a different location within the image.

Additionally, you can use the "Crop" option in the toolbar at the top of the screen to access additional settings for the crop tool. From there, you can choose whether you want to crop to a fixed ratio or enter custom dimensions, as well as adjust other options such as the aspect ratio and the position of the crop.

Do these instructions help clarify how to use the crop tool? If you have any more questions or need

{'input': "I don't know how to adjust the size and position of the crop ",
 'chat_history': [HumanMessage(content="Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool.", additional_kwargs={}, example=False),
  AIMessage(content=" Hello! I'd be happy to help you with the crop tool in our photo editing app! Could you please provide more details about what specifically is confusing you? For example, are you having trouble understanding how to select the area to crop, or how to adjust the size and position of the crop? Additionally, if you have any screenshots or examples of what you're trying to achieve, that would be very helpful for me to better understand your needs.", additional_kwargs={}, example=False)],
 'output': ' AI:  Sure thing! To adjust the size and position of the crop, you can use the handles that appear when you select an area to crop. The handles are the small squares that appear at the corners of the selection, and 

In [12]:
agent("Thanks that worked! Could you also guide me on how to adjust the brightness and contrast of an image obtained above?")



> Entering new AgentExecutor chain...
 AI:  Of course! Adjusting the brightness and contrast of an image can help bring out the details and improve the overall look of the photo.

To adjust the brightness and contrast of an image using our photo editing app, follow these steps:

1. Select the image you want to edit.
2. Click on the "Edit" button in the toolbar.
3. In the "Adjustments" panel, click on the "Brightness and Contrast" tab.
4. Use the sliders to adjust the brightness and contrast of the image.
5. To adjust the brightness, slide the brightness slider to the left to darken the image, or to the right to lighten it.
6. To adjust the contrast, slide the contrast slider to the left to reduce the difference between light and dark areas of the image, or to the right to increase the difference.
7. As you adjust the brightness and contrast, you can preview the changes in real-time by clicking on the "Preview" button.
8. When you're satisfied with the results, click on the "Done" but

{'input': 'Thanks that worked! Could you also guide me on how to adjust the brightness and contrast of an image obtained above?',
 'chat_history': [HumanMessage(content="Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool.", additional_kwargs={}, example=False),
  AIMessage(content=" Hello! I'd be happy to help you with the crop tool in our photo editing app! Could you please provide more details about what specifically is confusing you? For example, are you having trouble understanding how to select the area to crop, or how to adjust the size and position of the crop? Additionally, if you have any screenshots or examples of what you've tried so far, that would be really helpful for me to understand your issue better. I'll do my best to provide clear and concise instructions to help you master the crop tool!", additional_kwargs={}, example=False),
  HumanMessage(content="I don't know how to adjust the size and position of the crop